In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon

import geopandas as gpd

import folium

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

In [2]:
from pyproj import Transformer

# WGS84 (lat/lon) <-> Web Mercator (meters)
to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
to_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def project_to_3857(lon, lat):
    return to_3857.transform(lon, lat)

def project_to_4326(x, y):
    return to_4326.transform(x, y)

In [3]:
def bezier_curve(start, control1, control2, end, n=100):
    """Generate a cubic Bezier curve."""
    t = np.linspace(0, 1, n)[:, None]  # shape (n, 1)
    start, control1, control2, end = map(np.array, [start, control1, control2, end])
    curve = (
        (1 - t)**3 * start
        + 3 * (1 - t)**2 * t * control1
        + 3 * (1 - t) * t**2 * control2
        + t**3 * end
    )
    return curve


In [4]:
import numpy as np
from shapely.geometry import Polygon

def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left

def flow_patch_geo(
    start_ll, end_ll,
    start_width, end_width,
    bend=0.2,
    curvature=150,
    inherit_normal=None,   # optional: shared normal for siblings
):
    # ---- Project lon/lat to meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector
    d = end - start

    # ---- Inline 'unit' function ----
    d_norm = np.linalg.norm(d)
    if d_norm == 0:
        raise ValueError("Start and end coordinates are identical; cannot compute direction.")
    ud = d / d_norm  # unit direction vector

    # Perpendicular (normal) direction
    if inherit_normal is None:
        n = perp(ud)
    else:
        # Use parent's/trunk's normal, but normalize it
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # Control points
    ctrl1 = start + d * bend + n * curvature
    ctrl2 = end   - d * bend + n * curvature

    # Build Bezier edges
    top = bezier_curve(
        start + n * (start_width/2),
        ctrl1 + n * (start_width/2),
        ctrl2 + n * (end_width/2),
        end   + n * (end_width/2)
    )

    bottom = bezier_curve(
        end   - n * (end_width/2),
        ctrl2 - n * (end_width/2),
        ctrl1 - n * (start_width/2),
        start - n * (start_width/2)
    )

    # Combine vertices
    verts = np.vstack([top, bottom, top[0]])

    # Convert back to lon/lat
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    # Return polygon + this flow's normal for children
    return Polygon(verts_ll), n

In [5]:
## make a list of points 


## E line going from south to north
point_list = [
     (47.683729430644014, -122.34433918947462),
     (47.687259045590366, -122.34435249866571),
     (47.69108874059965, -122.34437859849116),
]

In [6]:
#ok I am going to make a short array describing the ons/offs and such

## first column is stop num, second column is on, third column is off.
rider_numbers = {
    0: [5, 0], #5
    1: [3, 2], #6
    2: [2, 4]  #4
}

In [7]:
Starting_Y_Val = point_list[0][0]
Second_Y_Val = point_list[1][0]
Third_Y_Val = point_list[2][0]

Starting_X_Val = point_list[0][1]
Second_X_Val = point_list[1][1]
Third_X_Val = point_list[2][1]

In [11]:

offset_const = 0.0003 ## this is the number that dictates 
width_const = 20 ## this number is to make the width visually look good while maintaining the on/off numbers

## initial main trunk
trunk_poly, trunk_normal = flow_patch_geo(
#Z_line_polygons.append(flow_patch_geo(
    start_ll = (Starting_X_Val, Starting_Y_Val),
    end_ll = (Second_X_Val, Second_Y_Val),
    start_width= 5 * width_const,
    end_width= 5 * width_const,
    bend=0.1,
    curvature=2,
)

##initial boarding
entry_branch_poly, entry_branch_normal = flow_patch_geo(
#Z_line_polygons.append(flow_patch_geo(
    start_ll = (Starting_X_Val - offset_const * 8, Starting_Y_Val),
    end_ll = (Second_X_Val - offset_const * 4, Second_Y_Val),
    start_width = 3 * width_const,
    end_width = 3 * width_const,
    bend= 0.2,
    curvature = -50,
    inherit_normal=trunk_normal,   # <<--- KEY FIX
)

#second main trunk
trunk_poly_2,trunk_normal_2 = flow_patch_geo(
    start_ll = (Second_X_Val, Second_Y_Val),
    end_ll = (Third_X_Val, Third_Y_Val),
    start_width=6 * width_const,
    end_width=6 * width_const,
    bend=0.2,
    curvature=20,
)

##second exit 
exit_branch_poly, exit_branch_normal = flow_patch_geo(
#Z_line_polygons.append(flow_patch_geo(
    start_ll = (Second_X_Val + offset_const * 2, Second_Y_Val),
    end_ll = (Third_X_Val + offset_const* 10, Third_Y_Val),
    start_width=2 * width_const,
    end_width=2 * width_const,
    bend=0.7,
    curvature=50,
    inherit_normal=trunk_normal_2,   # <<--- KEY FIX
)


polygon_list = [trunk_poly, entry_branch_poly, trunk_poly_2, exit_branch_poly]

In [12]:
colors = ['cornflowerblue', 'orange', 'cornflowerblue', 'green']

In [13]:
m = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=14)

for poly, color in zip(polygon_list, colors):
    folium.GeoJson(
        poly,
        style_function=lambda _, col = color: {
            "fillColor": col,
            "color": col,
            "weight": 1,
            "fillOpacity": 0.7
        }
    ).add_to(m)

m